In [1]:
import numpy as np
import cv2
import os

In [2]:
def crop_img(img):
	"""
	Finds the extreme points on the image and crops the rectangular out of them
	"""
	gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
	gray = cv2.GaussianBlur(gray, (5, 5), 0)

	# Otsu's thresholding - automatically finds optimal threshold value
	thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]
	thresh = cv2.erode(thresh, None, iterations=2)
	thresh = cv2.dilate(thresh, None, iterations=2)

	# find contours in thresholded image, then grab the largest one
	cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
	cnts = cnts[0] if len(cnts) == 2 else cnts[1]  # Replace imutils.grab_contours
	c = max(cnts, key=cv2.contourArea)

	# find the extreme points
	extLeft = tuple(c[c[:, :, 0].argmin()][0])
	extRight = tuple(c[c[:, :, 0].argmax()][0])
	extTop = tuple(c[c[:, :, 1].argmin()][0])
	extBot = tuple(c[c[:, :, 1].argmax()][0])
	ADD_PIXELS = 0
	new_img = img[extTop[1]-ADD_PIXELS:extBot[1]+ADD_PIXELS, extLeft[0]-ADD_PIXELS:extRight[0]+ADD_PIXELS].copy()
	
	return new_img

In [6]:
TRAIN = "./data/training/"
TEST = "./data/testing/"
TRAIN_CROPPED_PATH = "./data/cropped-otsu/training/"
TEST_CROPPED_PATH = "./data/cropped-otsu/testing/"
IMG_SIZE = 256

train_dir = os.listdir(TRAIN)
test_dir = os.listdir(TEST)

for dir in train_dir:
    save_path = TRAIN_CROPPED_PATH + dir
    path = os.path.join(TRAIN, dir)
    image_dir = os.listdir(path)
    for img_name in image_dir:
        img = cv2.imread(os.path.join(path, img_name))
        new_img = crop_img(img)
        new_img = cv2.resize(new_img, (IMG_SIZE, IMG_SIZE))
        if not os.path.exists(save_path):
            os.makedirs(save_path)
        cv2.imwrite(save_path + '/' + img_name, new_img)

for dir in test_dir:
    save_path = TEST_CROPPED_PATH + dir
    path = os.path.join(TEST, dir)
    image_dir = os.listdir(path)
    for img_name in image_dir:
        img = cv2.imread(os.path.join(path, img_name))
        new_img = crop_img(img)
        new_img = cv2.resize(new_img, (IMG_SIZE, IMG_SIZE))
        if not os.path.exists(save_path):
            os.makedirs(save_path)
        cv2.imwrite(save_path + '/' + img_name, new_img)